imports

In [1]:
import pandas as pd
import networkx as nx
from py2neo import Graph, Node
import json 

Load the data

In [2]:
courses_df = pd.read_excel("data/courses.xlsx")
students_df = pd.read_excel("data/students.xlsx")
completions_df = pd.read_excel("data/completions.xlsx")
registrations_df = pd.read_excel("data/registrations.xlsx")
course_description_df = pd.read_excel("data/courses_key_desc.xlsx")
course_outcomes_df = pd.read_excel("data/courses_key_outcomes.xlsx")
course_material_df = pd.read_excel("data/courses_key_material.xlsx")
courses_majors_df = pd.read_excel("data/courses_majors.xlsx")

clean the data and preprocess

In [3]:
# only include plan courses in the completed courses dataset
valid_course_ids = set(courses_df['course_id'].astype(str))
filtered_completions_df = completions_df[completions_df['course_id'].astype(str).isin(valid_course_ids)]

# credit hours as int
courses_df['credit_hours'] = courses_df['credit_hours'].astype(int)

# all text to lower case
course_description_df['topic'] = course_description_df['topic'].str.lower()
course_outcomes_df['topic'] = course_outcomes_df['topic'].str.lower()
course_material_df['topic'] = course_material_df['topic'].str.lower()

courses_df['outcomes'] = courses_df['outcomes'].str.lower()

list of topics from the 3 dataframes

In [4]:
# get list of topics 
combined_df = pd.concat([course_description_df, course_outcomes_df, course_material_df], ignore_index=True)
distinct_topics_array = combined_df['topic'].unique()
distinct_topics = pd.DataFrame(distinct_topics_array, columns=['topic_name'])

# Create a mapping of topics to their index
topic_to_index = {topic: idx for idx, topic in enumerate(distinct_topics['topic_name'])}

# Map the topic index back to the original dataframes
course_description_df['topic_id'] = course_description_df['topic'].map(topic_to_index)
course_outcomes_df['topic_id'] = course_outcomes_df['topic'].map(topic_to_index)
course_material_df['topic_id'] = course_material_df['topic'].map(topic_to_index)

get list of majors

In [5]:
majors_df = pd.DataFrame( courses_majors_df['major_id'].unique(), columns=['major_id'])

In [6]:
majors_df

major_id
0       CS
1       IS
2       IT

create the graph

In [23]:
# Create a graph
G = nx.DiGraph()

In [24]:
from dotenv import load_dotenv
import os

load_dotenv()

neo4j_host = os.getenv("NEO4J_HOST")
neo4j_password = os.getenv("NEO4J_PASS")

# Connect to Neo4j
graph = Graph(neo4j_host, auth=("neo4j", neo4j_password))

delete all ( if a graph has been created and you want to start all over again )

In [25]:
graph.run("MATCH (n) DETACH DELETE n")

(No data)

delete all relationships

In [ ]:
# delete all relationships 
graph.run("MATCH ()-[r]->() DELETE r")

delete a node type with its relationships

In [ ]:
graph.run("MATCH (t:Topic)-[r]-() DELETE r, t")

delete all relationships connected to a student

In [ ]:
def delete_student_relationships(student_id):
    query = """
    MATCH (s:Student {id: $studentId})-[r]-() 
    DELETE r
    """
    graph.run(query, studentId=student_id)
    print(f"Deleted all relationships connected to student with id {student_id}.")
delete_student_relationships('201801657')

add nodes

In [26]:
# Add courses to the graph
for _, row in courses_df.iterrows():
    node_data = {
        "id": row['course_id'],
        "name": row['name'],
        "credit_hours": row['credit_hours'], 
        "type": row['type']
    }
    
    node = Node("Course", **node_data)
    G.add_node(row['course_id'], **node_data)
    graph.create(node)

In [27]:
# Add students to the graph
for _, row in students_df.iterrows():
    node_data = {
        "id": str(row['id'])
    }

    node = Node("Student", **node_data)
    G.add_node(row['id'], **node_data)
    graph.create(node)

In [28]:
# Add topics to the graph
for index, row in distinct_topics.iterrows():
    node_data = {
        "id": f"topic_{index}", 
        "name": row['topic_name']
    }

    node = Node("Topic", **node_data)
    G.add_node("Topic", **node_data)
    graph.create(node)

In [29]:
# Add majors to the graph
for index, row in majors_df.iterrows():
    node_data = {
        "id": row['major_id'], 
        "name": row['major_id']
    }

    node = Node("Major", **node_data)
    G.add_node("Major", **node_data)
    graph.create(node)

add relationships to networkx graph

In [30]:
# Iterate over courses_df to add prerequisites and co-courses
for _, row in courses_df.iterrows():
    course_id = f"Course_{row['course_id']}"

    # Handle prerequisites
    if pd.notnull(row['prerequisites']):  # Ensure the column isn't NaN
        prerequisites = row['prerequisites'].split(',')  # Split by comma
        for prereq in prerequisites:
            prereq_id = f"Course_{prereq.strip()}"  # Strip any extra spaces
            G.add_edge(prereq_id, course_id, label="prerequisite_for")
            G.add_edge(course_id, prereq_id, label="requires")
    
    # Handle co-courses
    if pd.notnull(row['co_courses']):  # Ensure the column isn't NaN
        co_courses = row['co_courses'].split(',')  # Split by comma
        for co_course in co_courses:
            co_course_id = f"Course_{co_course.strip()}"  # Strip any extra spaces
            G.add_edge(course_id, co_course_id, label="co_course")
            G.add_edge(co_course_id, course_id, label="co_course")

# Filtered completions
for _, row in filtered_completions_df.iterrows():
    G.add_edge(
        f"Student_{row['student_id']}",
        f"Course_{row['course_id']}",
        label="completed",
        grade=row['grade'],
        term_sequence=row['term_sequence']
    )
    G.add_edge(
        f"Course_{row['course_id']}",
        f"Student_{row['student_id']}",
        label="completed_by",
        grade=row['grade'],
        term_sequence=row['term_sequence']
    )


# Registration
for _, row in registrations_df.iterrows():
    G.add_edge(f"Student_{row['student_id']}",  f"Course_{row['course_id']}",  label="registered_for")
    G.add_edge(f"Course_{row['course_id']}" , f"Student_{row['student_id']}", label="registered_by")

In [31]:
# Add Course-Topic relationships for course descriptions
for _, row in course_description_df.iterrows():
    topic_node = f"Topic_{row['topic_id']}" 
    course_node = f"Course_{row['course_id']}" 
    G.add_edge(course_node, topic_node, label="description_topic") 
    G.add_edge(topic_node, course_node, label="description_topic")

# Add Course-Topic relationships for course outcomes
for _, row in course_outcomes_df.iterrows():
    topic_node = f"Topic_{row['topic_id']}"
    course_node = f"Course_{row['course_id']}"
    G.add_edge(course_node, topic_node, label="outcome_topic")
    G.add_edge(topic_node, course_node, label="outcome_topic")

# Add Course-Topic relationships for course materials
for _, row in course_material_df.iterrows():
    topic_node = f"Topic_{row['topic_id']}"
    course_node = f"Course_{row['course_id']}"
    G.add_edge(course_node, topic_node, label="material_topic")
    G.add_edge(topic_node, course_node, label="material_topic")

In [32]:
# majors
for _, row in courses_majors_df.iterrows():
    G.add_edge(f"Major_{row['major_id']}", f"Course_{row['course_id']}", label="has_course")
    G.add_edge(f"Course_{row['course_id']}" , f"Major_{row['major_id']}", label="for_major")

# students to majors 
for _, row in students_df.iterrows():
    G.add_edge(f"Major_{row['major_id']}", f"Student_{row['id']}", label="has_student")
    G.add_edge(f"Student_{row['id']}" , f"Major_{row['major_id']}", label="majored_in")

create index for better query performance

In [ ]:
graph.run("CREATE INDEX FOR (n:Course) ON (n.id)")
graph.run("CREATE INDEX FOR (n:Student) ON (n.id)")
graph.run("CREATE INDEX FOR (n:Topic) ON (n.id)")

add relationships to neo4j graph

In [34]:
for source, target, edge_data in G.edges(data=True):
    # Determine the label of the source node
    if source.startswith("Course"):
        a_label = "Course"
        source_id = source.split("_")[1]  # Extract course ID
    elif source.startswith("Student"):
        a_label = "Student"
        source_id = source.split("_")[1]  # Extract student ID
    elif source.startswith("Topic"):
        a_label = "Topic"
        source_id = "topic_" + source.split("_")[1]  # Extract topic ID
    else:
        a_label = "Major"
        source_id = source.split("_")[1]  # Extract major ID

    # Determine the label of the target node
    if target.startswith("Course"):
        b_label = "Course"
        target_id = target.split("_")[1]  # Extract course ID
    elif target.startswith("Student"):
        b_label = "Student"
        target_id = target.split("_")[1]  # Extract student ID
    elif source.startswith("Topic"):
        b_label = "Topic"
        target_id = "topic_" + source.split("_")[1]  # Extract topic ID
    else:
        b_label = "Major"
        target_id = source.split("_")[1]  # Extract major ID

    # Create a MERGE query for relationships to avoid duplicates
    query = """
    MATCH (a:{a_label} {{id: $source_id}})
    MATCH (b:{b_label} {{id: $target_id}})
    MERGE (a)-[r:{rel_type}]->(b)
    ON CREATE SET r += $props
    RETURN CASE WHEN r IS NOT NULL THEN 'created' ELSE 'exists' END AS status
    """

    # Run the query and get the result
    result = graph.run(query.format(a_label=a_label, b_label=b_label, rel_type=edge_data['label']),
                       source_id=source_id, target_id=target_id, props=edge_data).data()

    # Check if the relationship was created and print a message
    if result and result[0]['status'] == 'created':
        print(f"Relationship {edge_data['label']} between {source} and {target} was created.")
    else: 
        print(f"Relationship {edge_data['label']} between {source} and {target} already exists.")

Relationship co_course between Course_CSBP121 and Course_CSBP119 was created.
Relationship completed_by between Course_CSBP121 and Student_201801657 was created.
Relationship completed_by between Course_CSBP121 and Student_201899887 was created.
Relationship completed_by between Course_CSBP121 and Student_201800871 was created.
Relationship completed_by between Course_CSBP121 and Student_201944556 was created.
Relationship description_topic between Course_CSBP121 and Topic_28 already exists.
Relationship description_topic between Course_CSBP121 and Topic_29 already exists.
Relationship material_topic between Course_CSBP121 and Topic_24 already exists.
Relationship material_topic between Course_CSBP121 and Topic_26 already exists.
Relationship description_topic between Course_CSBP121 and Topic_30 already exists.
Relationship material_topic between Course_CSBP121 and Topic_31 already exists.
Relationship material_topic between Course_CSBP121 and Topic_32 already exists.
Relationship desc

create level nodes

In [35]:
level_nodes = [Node("Level", id=str(i), name=f"Level {i}") for i in range(1, 5)]
for level_node in level_nodes:
    graph.create(level_node)

add level relationships

In [36]:

for _, row in courses_df.iterrows():
    course_id = row['course_id']
    level = course_id[4]  # Extract the 5th character for level
    if level == '0': 
        level = '1'

    # Create a unique identifier for the level node
    level_node_id = str(level)  # Use the level directly as ID (1, 2, 3, 4)

    # Check if the level node exists
    query_check_level = """
    MATCH (l:Level {id: $level_id})
    RETURN COUNT(l) AS count
    """
    level_exists = graph.run(query_check_level, level_id=level_node_id).evaluate()

    if level_exists == 0:
        print(f"Level node with ID {level_node_id} does not exist.")
        continue  # Skip to the next course if the level node doesn't exist

    # Create the relationship
    query = """
    MATCH (c:Course {id: $course_id}), (l:Level {id: $level_id})
    MERGE (c)-[:has_level]->(l)
    RETURN CASE WHEN l IS NOT NULL THEN 'created' ELSE 'exists' END AS status
    """

    # Run the query to create the relationship
    result = graph.run(query, course_id=course_id, level_id=level_node_id).data()

    # Check if the relationship was created and print a message
    if result and result[0]['status'] == 'created':
        print(f"Relationship HAS_LEVEL between {course_id} and Level {level} was created.")
    else:
        print(f"Relationship HAS_LEVEL between {course_id} and Level {level} already exists.")

Relationship HAS_LEVEL between MATH105 and Level 1 was created.
Relationship HAS_LEVEL between CHEM111 and Level 1 was created.
Relationship HAS_LEVEL between CSBP119 and Level 1 was created.
Relationship HAS_LEVEL between CSBP121 and Level 1 was created.
Relationship HAS_LEVEL between ESPU1081 and Level 1 was created.
Relationship HAS_LEVEL between GEIT112 and Level 1 was created.
Relationship HAS_LEVEL between MATH110 and Level 1 was created.
Relationship HAS_LEVEL between CSBP219 and Level 2 was created.
Relationship HAS_LEVEL between CSBP221 and Level 2 was created.
Relationship HAS_LEVEL between CENG205 and Level 2 was created.
Relationship HAS_LEVEL between CENG202 and Level 2 was created.
Relationship HAS_LEVEL between MATH140 and Level 1 was created.
Relationship HAS_LEVEL between CSBP319 and Level 3 was created.
Relationship HAS_LEVEL between CENG210 and Level 2 was created.
Relationship HAS_LEVEL between PHYS105 and Level 1 was created.
Relationship HAS_LEVEL between STAT210 

reset weights

In [37]:
queries = [
    
    # reset all weights
        """
    MATCH ()-[r]->()
    SET r.weight = 1.0
    """
]

for query in queries: 
    graph.run(query)
print("weights have been updated successfully")

weights have been updated successfully


In [38]:
queries = [
    
    # reset all weights
        """
    MATCH ()-[r]->()
    SET r.weight = 1.0
    """
    , 
    
    # Step 3: Update weights for `has_level` relationships
    """
    MATCH (c:Course)-[r:has_level]->(l:Level)
    SET r.weight = CASE 
        WHEN l.id = '1' THEN 50.0  // Highest
        WHEN l.id = '2' THEN 3.5
        WHEN l.id = '3' THEN 2.0
        WHEN l.id = '4' THEN 0.5  // Lowest
        WHEN l.id = '5' THEN 0.2  // Lowest
        ELSE 0.5  // Default (if level is missing)
    END
    """,
       # Step 1: Adjust weights for relationships involving level 1 courses
    """
    MATCH (c:Course)-[r]->()
    MATCH (c)-[:has_level]->(l:Level)
    SET r.weight = CASE 
        WHEN l.id = '1' THEN COALESCE(r.weight, 1.0) * 5  // Highest importance
        WHEN l.id = '2' THEN COALESCE(r.weight, 1.0) * 2.0
        WHEN l.id = '3' THEN COALESCE(r.weight, 1.0) * 1.5
        WHEN l.id = '4' THEN COALESCE(r.weight, 1.0) * 0.5  // Lowest importance
        WHEN l.id = '5' THEN COALESCE(r.weight, 1.0) * 0.2  // Lowest importance        
        ELSE COALESCE(r.weight, 1.0) * 0.5  // Default (if level is missing or undefined)
    END
    """,
    
    # Step 2: Adjust weights for relationships incoming to courses
    """
    MATCH ()-[r]->(c:Course)
    MATCH (c)-[:has_level]->(l:Level)
    SET r.weight = CASE 
        WHEN l.id = '1' THEN COALESCE(r.weight, 1.0) * 5  // Highest importance
        WHEN l.id = '2' THEN COALESCE(r.weight, 1.0) * 2.0
        WHEN l.idid = '3' THEN COALESCE(r.weight, 1.0) * 1.5
        WHEN l.id = '4' THEN COALESCE(r.weight, 1.0) * 0.5  // Lowest importance
        WHEN l.id = '5' THEN COALESCE(r.weight, 1.0) * 0.2  // Lowest importance
        ELSE COALESCE(r.weight, 1.0) * 0.5  // Default (if level is missing or undefined)
    END
    """
]

for query in queries: 
    graph.run(query)
print("weights have been updated successfully")

weights have been updated successfully


create in-memory graph to convert into embedding

In [41]:
def create_in_memory_graph(graph, graph_name='courseGraph', include_all_relationships=True, specific_relationships=None):
    # Check if the graph already exists
    query_check = f"""
    CALL gds.graph.exists('{graph_name}') YIELD exists
    """
    result = graph.run(query_check).evaluate()

    if result:  # If the graph exists, result will be True
        print(f"Graph '{graph_name}' already exists. Deleting it before creating a new one.")
        # Delete the existing graph
        query_drop = f"""
        CALL gds.graph.drop('{graph_name}')
        """
        graph.run(query_drop)
    
    # Create the in-memory graph
    if include_all_relationships:
        query_create = """
        CALL gds.graph.project(
           'courseGraph',
            ['Course', 'Student', 'Topic', 'Level', 'Major'],  // Node types to include
            {
                prerequisite_for: { type: 'prerequisite_for', properties: 'weight' },
                co_course: { type: 'co_course', properties: 'weight' },
                requires: { type: 'requires', properties: 'weight' },
                completed_by: { type: 'completed_by', properties: 'weight' },
              //  registered_for: { type: 'registered_for', properties: 'weight' },
                material_topic: { type: 'material_topic', properties: 'weight' },
                description_topic: { type: 'description_topic', properties: 'weight' },
                outcome_topic: { type: 'outcome_topic', properties: 'weight' },
                has_level: { type: 'has_level', properties: 'weight' },
                completed: { type: 'completed', properties: 'weight' }
            }
        )
        """

    elif specific_relationships:
        # Ensure specific_relationships is a list
        if isinstance(specific_relationships, list):
            relationship_types = ', '.join(f"'{relationship}'" for relationship in specific_relationships)
        else:
            raise ValueError("specific_relationships must be a list of relationship types.")
        
        query_create = f"""
        CALL gds.graph.project(
            '{graph_name}',
            '*' ,  // Project all nodes
            [{relationship_types}]  // Project specified relationships
        )
        """
    else:
        raise ValueError("Must provide either all relationships or specific relationships.")

    graph.run(query_create)
    print(f"Graph '{graph_name}' created successfully.")


In [42]:
create_in_memory_graph(graph)

Graph 'courseGraph' created successfully.
